Boltzmann Machine Recommender

Import the Libraries Needed

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

Import the Datasets

In [2]:
#Column 0 is Movie ID, 1 is the title, 2 is the genre
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
#Column 0 is user, 1 is gender, 2 is, 3 is, 4 is
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [4]:
#Column 0 is UserID, 1 is MovieID, 2 is the Ratings, 3 is timestamps and will be removed
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


Preparing the train and test set

In [6]:
#This contains 80k reviews of the 100k so this is 80% of the data
training_set = np.array(pd.read_csv('ml-100k/u1.base', delimiter='\t'), dtype='int')

In [7]:
test_set = np.array(pd.read_csv('ml-100k/u1.test', delimiter='\t'), dtype='int')

Getting the number of users and movies

In [9]:
#Get the max number among both training and test set for the amount of users and amount of movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

Convert the data into an array with users in lines and movies in columns

In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [12]:
training_set = convert(training_set)
test_set = convert(test_set)

Converting the data into Pytorch Tensors

In [15]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Converting the ratings into binary ratings (1 for liked or 0 for not liked)

In [16]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [17]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

Creating the architecture of the Neural Network

In [28]:
#Reference document AltRBM-proof.pdf to grasp the mathematical functions that are being coded below

class RBM():
    #When this class is initialized the first pass will initialize random weights and bias
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
        
    #Generating the probability of a hidden node given the probability of the visible node THEN:
    #Sample the activation of the hidden nodes
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    #Generating the probability of a visible node given the probability of the hidden node THEN:
    #Sample the activation of the visible nodes
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
#Refer to section 5 'Approximating the RBM Log-Likelihood Gradient' Updating values for hidden and visible nodes
#Simply coding the equation from the k-step contrastive divergence technique (page 28) in the research paper
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)        

In [29]:
# Defining the number of features for the visible nodes which is the ratings the user gave each movie
nv = len(training_set[0])

#Defining the number of hidden nodes (which is up to us architecturally) this can be adjusted to tune the model
#I have arbitrally decided to scan the input for 100 features 
#These will attempt to explain the likelihood the user will like a movie
nh = 100

#Update the weights in the model after a certain number of observations defined as 'batches'
#This is arbitrary and tuneable
batch_size = 100

In [30]:
rbm = RBM(nv, nh)

Training the RBM

In [33]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: ' +str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.2445)
epoch: 2 loss: tensor(0.2450)
epoch: 3 loss: tensor(0.2471)
epoch: 4 loss: tensor(0.2456)
epoch: 5 loss: tensor(0.2512)
epoch: 6 loss: tensor(0.2468)
epoch: 7 loss: tensor(0.2488)
epoch: 8 loss: tensor(0.2485)
epoch: 9 loss: tensor(0.2466)
epoch: 10 loss: tensor(0.2472)


Testing the RBM

In [34]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2533)
